In [1]:
%load_ext autoreload
%autoreload 2
%cd ~/austin_big_vision
import io
import os
import jax
import json
import torch
import requests
import importlib
import numpy as np
from PIL import Image
import ml_collections
import jax.numpy as jnp
import tensorflow as tf

import open_clip
import big_vision.utils as u
import big_vision.models.vit as model_mod
from big_vision.models.vit import scan_to_pyloop

/home/austinwang/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/home/austinwang/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/home/austinwang/austin_big_vision


2024-10-26 02:30:17.501005: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-26 02:30:17.523545: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-26 02:30:17.530370: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-26 02:30:18.568260: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
backbone_path = 'gs://us-central2-storage/tensorflow_datasets/vit-b-16_3b_pretraining/coca_replication_s3b_bs16384_warm0.03_1.0co-1.0ca_lr1e-3_wd1e-4_bf16_qknorm-F_b2-0.95_6lyr_scan-F_fsdp-F_08-19_0355'
config_path = f'{backbone_path}/config.json'
metrics_path = f'{backbone_path}/big_vision_metrics.txt'
config = ml_collections.ConfigDict(json.load(tf.io.gfile.GFile(config_path, 'r')))
bv_metrics = json.loads(tf.io.gfile.GFile(metrics_path, 'r').read().split('\n')[-2])
ckpt_path = f'{backbone_path}/checkpoint.bv-{config.total_steps:09d}'

In [3]:
model_cfg = config.model
img_key = 'img' if 'image' in model_cfg or 'img' in model_cfg else 'encoder'

# load model
model_mod = importlib.import_module(f"big_vision.models.{config.model_name}")
bv_model = model_mod.Model(**model_cfg)

# load ckpt weights
rng = jax.random.PRNGKey(42)
dummy_img = jnp.zeros([2, 224, 224, 3], jnp.float32)
dummy_txt = jnp.zeros([2, 64], jnp.int32)
dummy_mask_ar = jnp.zeros([2, 64], jnp.bool_) if 'llm' in model_cfg else None
if dummy_mask_ar is not None:
    init_params = jax.jit(bv_model.init, backend="cpu")(rng, dummy_img, dummy_txt,dummy_mask_ar)['params']
else:
    init_params = jax.jit(bv_model.init, backend="cpu")(rng, dummy_img, dummy_txt)['params']
# img_load_kw = {'dont_load': ('.*_ln/scale','head/kernel', 'head/bias')}
params = model_mod.load(init_params, ckpt_path, model_cfg) #, img_load_kw)
# jax.tree_map(jnp.shape, img_params)
jax.tree_map(jnp.dtype, params)

: 

In [ ]:

url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/tasks/car.jpg?download=true"
image = Image.open(requests.get(url, stream=True).raw)

In [ ]:
bv_img = 
bv_out = bv_model.apply({"params":params}, bv_img, return_enc_features=True, method = 'encode')